Optimizing

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import os
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report


/Users/drapcat/4380env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
# Constants
IMG_SIZE = 224
BATCH_SIZE = 16
base_dir = "Data"

train_dir = os.path.join(base_dir, "train")
valid_dir = os.path.join(base_dir, "valid")
test_dir = os.path.join(base_dir, "test")

# Data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomBrightness(0.1)
])

# Load datasets
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode="binary",
    shuffle=True
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    valid_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode="binary",
    shuffle=False
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode="binary",
    shuffle=False
)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)
test_ds = test_ds.prefetch(AUTOTUNE)


Found 280 files belonging to 2 classes.
Found 59 files belonging to 2 classes.
Found 59 files belonging to 2 classes.


In [4]:
# Using ResNet50 as base
base_model = tf.keras.applications.ResNet50(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # Try frozen base first

# Build model
inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model = models.Model(inputs, outputs)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 8s 0us/step


In [5]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=[early_stopping]
)


Epoch 1/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 10s 424ms/step - accuracy: 0.4693 - loss: 0.8447 - val_accuracy: 0.6441 - val_loss: 0.6715
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 7s 394ms/step - accuracy: 0.5913 - loss: 0.7534 - val_accuracy: 0.6441 - val_loss: 0.6425
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 7s 410ms/step - accuracy: 0.5270 - loss: 0.7214 - val_accuracy: 0.6441 - val_loss: 0.6450
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 7s 408ms/step - accuracy: 0.6336 - loss: 0.7024 - val_accuracy: 0.6441 - val_loss: 0.6365
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 7s 406ms/step - accuracy: 0.6438 - loss: 0.6261 - val_accuracy: 0.6441 - val_loss: 0.6337
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 8s 423ms/step - accuracy: 0.6317 - loss: 0.6601 - val_accuracy: 0.6441 - val_loss: 0.6296
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 8s 421ms/step - accuracy: 0.6365 - loss: 0.6426 - val_accuracy: 0.6441 - val_loss: 0.6278
Epoch 8/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 8s 425ms/step - accuracy: 0.6791 - loss: 0.6356 - val_accuracy: 0

In [6]:
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test Accuracy: {test_acc:.2f}")


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 273ms/step - accuracy: 0.6139 - loss: 0.6223
Test Accuracy: 0.78


In [7]:
y_true = []
y_pred = []

for images, labels in test_ds:
    probs = model.predict(images).flatten()
    preds = (probs > 0.5).astype(int)
    y_true.extend(labels.numpy())
    y_pred.extend(preds)

# Display confusion matrix and classification report
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=["Not Fallen", "Fallen"]))


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 803ms/step
Confusion Matrix:
[[ 2 13]
 [ 0 44]]

Classification Report:
              precision    recall  f1-score   support

  Not Fallen       1.00      0.13      0.24        15
      Fallen       0.77      1.00      0.87        44

    accuracy                           0.78        59
   macro avg       0.89      0.57      0.55        59
weighted avg       0.83      0.78      0.71        59



2025-07-20 23:47:33.263275: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [8]:
base_model.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # LOWER learning rate
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [12]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    restore_best_weights=True
)

history_finetune = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[early_stopping]
)


Epoch 1/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.7023 - loss: 0.7932 - val_accuracy: 0.6441 - val_loss: 0.8881
Epoch 2/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step - accuracy: 0.7133 - loss: 0.7545 - val_accuracy: 0.6441 - val_loss: 1.6300
Epoch 3/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.7250 - loss: 0.5350 - val_accuracy: 0.6441 - val_loss: 2.7453
Epoch 4/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - accuracy: 0.7756 - loss: 0.4503 - val_accuracy: 0.6441 - val_loss: 2.5299


In [13]:
y_true = []
y_pred = []

for images, labels in test_ds:
    probs = model.predict(images).flatten()
    preds = (probs > 0.5).astype(int)
    y_true.extend(labels.numpy())
    y_pred.extend(preds)

# Display confusion matrix and classification report
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=["Not Fallen", "Fallen"]))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 341ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
Confusion Matrix:
[[ 0 15]
 [ 0 44]]

Classification Report:
              precision    recall  f1-score   support

  Not Fallen       0.00      0.00      0.00        15
      Fallen       0.75      1.00      0.85        44

    accuracy                           0.75        59
   macro avg       0.37      0.50      0.43        59
weighted avg       0.56      0.75      0.64        59



/Users/drapcat/4380env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/drapcat/4380env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/drapcat/4380env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [14]:
# Count how many images per class in training set
fallen_count = len(os.listdir(os.path.join(train_dir, 'fallen')))
not_fallen_count = len(os.listdir(os.path.join(train_dir, 'not_fallen')))
print(f"Fallen: {fallen_count}, Not Fallen: {not_fallen_count}")


Fallen: 97, Not Fallen: 183
